<h1 style="color:red";> Our research question</h1>

<p>How many citations (according to <a href="http://opencitations.net/download#coci"><b>COCI</b></a>) involve, either as citing or cited entities, publications in SSH journals (according to <a href="https://kanalregister.hkdir.no/publiseringskanaler/erihplus/periodical/listApproved"><b>ERIH-PLUS</b></a>) included in <b>OpenCitations Meta</b>? What are the disciplines that cites the most and those cited the most? How many citations start from and go to publications in OpenCitations Meta that are not included in SSH journals?</p>


<h2 style="color:red;">27/03</h2>

<p>Today I started thinking about the abstract and first of all I have read <a href="https://www.emeraldgrouppublishing.com/how-to/authoring-editing-reviewing/write-article-abstract#structured-abstract">the guideline</a> proposed by Emerald Publishing to sketch it.</p>
<p><b>Maximum amout of words</b>: "most journal author guidelines set a maximum of 250 words, including keywords and article classification".</p>

<p>In order to try to figure out the methodology to follow to answer to our rearch question, I thought it could have been useful to start exploring the <b>structure of the data</b>. I've downloaded the list of approved journals in ERIH-PLUS in the csv format and also one archive of COCI's dump.</p>

<h4>ERIH PLUS data</h4>

In [1]:
import pandas as pd
df_ErihPlus = pd.read_csv("2023-03-27 ERIH PLUS approved journals and series.csv", sep=";")

In [2]:
df_ErihPlus.columns

Index(['Journal ID', 'Print ISSN', 'Online ISSN', 'Original Title',
       'International Title', 'Country of Publication',
       'ERIH PLUS Disciplines', 'OECD Classifications', '[Last Updated]'],
      dtype='object')

In [3]:
df_ErihPlus.dtypes

Journal ID                 int64
Print ISSN                object
Online ISSN               object
Original Title            object
International Title       object
Country of Publication    object
ERIH PLUS Disciplines     object
OECD Classifications      object
[Last Updated]            object
dtype: object

<h4>COCI data</h4>

In [4]:
import zipfile
from os.path import exists

def extract_file(input_dir, output_dir):
    if not output_dir:
            if not exists(output_dir):
                makedirs(output_dir)
    with zipfile.ZipFile(input_dir, 'r') as zip_ref:
        zip_ref.extractall(output_dir)    

In [5]:
extract_file("2022-12-31T201913_0-19_1.zip", "COCI_data")

<p>In each csv there are 7 columns: oci, citing (doi), cited (doi), creation (date), timespan, journal_sc (yes or no), author_sc(yes or no). The last two columns provide the information to know if the citing and cited entities share at least one author and if they have been published in the same journal.</p>
<p>OCI is the OpenCitation identifier for the citation.</p>